In [ ]:
import numpy as np
import rospy
import baxter_interface
import pickle

import copy
from motion_planning_data_gen import *

### Set up functions for collision checking and playback

In [ ]:
# Joint ranges for the baxter, in the order defined in the Baxter SDK (NOT the MoveIt! order)
joint_ranges = np.array([3.4033, 3.194, 6.117, 3.6647, 6.117, 6.1083, 2.67])

def compute_cost(path):
    # Overall Euclidean path length in C-space
    dlist = []
    for k in range(len(path)-1):
        sq_dists = 0
        for j in range(0,7):
            diff = path[k][j] - path[k+1][j]
            sq_dist += diff * diff

        dlist.append(math.sqrt(sq_dists))

    return sum(dlist)

def unscramble_path(path):
    # Calls moveit_unscrambler function on each state to change from MoveIt joint order to Baxter joint order
    new_path = []
    for state in path:
        new_state = moveit_unscrambler(state)
        new_path.append(new_state)
        
    return new_path

def check_full_path(overall_path):
    # Calls a state validity service on each state in the path
    invalid = []

    valid = True           #state validity
    overall_valid = True   #overall validity

    for i, state in enumerate(overall_path):
        filler_robot_state[10:17] = moveit_scrambler(state)
        rs_man.joint_state.position = tuple(filler_robot_state)
        collision_free = sv.getStateValidity(rs_man, group_name="right_arm")

        valid = valid and collision_free
        overall_valid = overall_valid and collision_free

        if not valid:
            invalid.append(i)
            valid = True    #reset on each waypoint

    if (len(invalid)==0 and overall_valid):
        print("Full path valid!")
    else:
        print("Not valid")
        
    return overall_valid

def play_smooth(overall_path, sleep=False):
    k = 1
    
    joint_state = limb.joint_angles()
    if sleep:
        print("sleeping for 5 seconds to setup")
        time.sleep(5)
    
    done = False

    while not done:
        for i, name in enumerate(joint_state.keys()):
            joint_state[name] = overall_path[k][i]
        there = False
        start = time.time()
        while not there:
            limb.set_joint_positions(joint_state)
            now = time.time()
            if (now - start) > 0.05:
                there = True
        k += 1
        if k > len(overall_path)-1:
            done = True

### Set up ROS node, baxter interface, and planning scene, and load environment information

In [ ]:
rospy.init_node("playback")

limb_connect = False
while not limb_connect:
    try:
        limb = baxter_interface.Limb('right')
        limb_connect = True
    except OSError:
        limb_connect = False
        
limb.set_joint_position_speed(0.25)        
gripper = baxter_interface.Gripper('right')

scene = PlanningSceneInterface()
scene._scene_pub = rospy.Publisher('planning_scene', PlanningScene, queue_size=0)
robot = RobotCommander()
group = MoveGroupCommander("right_arm")
sv = StateValidity()

set_environment(robot, scene)

with open("env/trainEnvironments.pkl", "rb") as env_f:
    masterEnvDict = pickle.load(env_f)

# Load the scene with the obstacles as defined in the environment dictionary file, under key 'obsData'
# which contains the obstacle information (mesh file, scale, name, etc.)
sceneModifier = PlanningSceneModifier(masterEnvDict['obsData'])
sceneModifier.setup_scene(scene, robot, group)

# Construct a state object in case you want to do collision checking
rs_man = RobotState()
robot_state = robot.get_current_state()
rs_man.joint_state.name = robot_state.joint_state.name
filler_robot_state = list(robot_state.joint_state.position)

### Apply changes to environment

In [ ]:
sceneModifier.delete_obstacles()

In [ ]:
# all environments are called 'trainEnv_N' with N = linspace(0, 9, 1)
env = 'trainEnv_4'
new_pose = masterEnvDict['poses'][env]
sceneModifier.permute_obstacles(new_pose)

### Open path data file, set up for playback, and use Baxter interface to execute

In [ ]:
with open('data/path_data_example_' + env + '.pkl', 'rb') as path_f:
    paths_dict = pickle.load(path_f)   

In [ ]:
path = paths_dict['paths'][0][1:] # all feasible paths saved in paths_dict['paths'] list, first state duplicate removed
path = unscramble_path(path) # MoveIt and baxter_interface order joints differently

path_rev = copy.deepcopy(path)
path_rev.reverse()

check_full_path(path)

In [ ]:
play_smooth(path)